<a href="https://colab.research.google.com/github/ialara/actf/blob/main/absorption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# F-16 Absorption Model
## Ignacio Lara

### Steps
1. Initialization
  1. Populate the squadron
    * Size (API-1 and API-6)
    * Distribution of INX/EXP
    * Distribution of sortie counts/qualifications/placement in upgrades
1. Inflow new pilots
  * Distribution of B-Course vs. FAIP/PQP (including qualifications of nth-tour)
  * Distribution of API-1 vs API-6
1. Determine resources available
   * Flying Hours remaining
   * Sorties (capacity, driven by UTE and PAA)
   * IPs and/or appropriate upgrade support pilots
1. Determine sortie needs
   * Upgrade vs. CT
   * ASD for FHP accrual
1. Schedule resources for sorties
1. Attempt to fly sorties
   * Refly rate to capture ground abort, incomplete sorties, wx cancellations, etc.
   * Decrement FHP for sorties flown
1. Credit sortie completion for syllabus and/or RTM "beans"
1. Award upgrades / experiencing
1. Outflow pilots

<hr />

### 0. Setup

In [ ]:
True != True

False

In [39]:
import numpy as np

rng = np.random.default_rng()

ug_names = ['MQT', 'FLUG', 'IPUG']
ug_rides = [10,     15,     8]
ug_quals = ['WM',   'FL',   'IP']

class Pilot:
    """An Air Force F-16 pilot that fills squadron billets"""
    # Syllabus lengths (number of UP sorties)
    syllabi_rides = {ug: dur for ug, dur in zip(ug_names, ug_rides)}

    # Quals awarded when completing upgrades
    ug_awards = {ug: q for ug, q in zip(ug_names, ug_quals)}

    # Everyone gets these many sorties from FTU
    FTU_sorties = 15

    # Definition of experience
    exp_sorties = 250
    exp_qual = 'FL'

    def __init__(self, id, f16_sorties, tos, api_category, quals=[], ug=None):
        self.id = id
        self.f16_sorties = self.FTU_sorties + f16_sorties
        self.tos = tos
        self.quals = quals
        self.is_exp = False
        self.check_experience()
        self.api_category = 1 if not self.is_exp else api_category
        self.ug = None
        if ug is not None:
            self.enroll_upgrade(ug)

    def log(self, msg, prefix='>>'):
        print(f'{prefix} PID {self.id} {msg}')

    def enroll_upgrade(self, ug):
        assert self.ug is None, 'Pilot already enrolled in upgrade'
        assert ug in ug_names, 'Invalid upgrade specified'
        assert self.ug_awards[ug] not in self.quals, 'Pilot already completed this upgrade'
        self.ug = ug
        self.ride_num = 0
        self.pending_qual = self.ug_awards[self.ug]
        self.log(f'enrolled in {self.ug}')

    def disenroll_upgrade(self):
        prev_ug = self.ug
        self.ug = None
        self.log(f'disenrolled from: {prev_ug}', prefix='<<')

    def fly_ug_sortie(self):
        assert self.ug is not None, 'Pilot not enrolled in upgrade'
        self.ride_num += 1
        if self.check_ug_complete(self.ug, self.ride_num):
          self.award_qual(self.pending_qual)
          self.disenroll_upgrade()
        self.fly_sortie()

    def check_ug_complete(self, ug, ride):
        return ride == self.syllabi_rides[ug]

    def fly_sortie(self):
        self.f16_sorties += 1
        self.check_experience()

    def award_qual(self, qual):
        assert qual in ug_quals, 'Invalid qualification specified'
        self.quals.append(qual)
        self.log(f'awarded: {qual}', prefix='++')

    def check_experience(self):
        prev_status = self.is_exp
        self.is_exp = (self.f16_sorties >= self.exp_sorties and 
                      self.exp_qual in self.quals)
        if self.is_exp != prev_status:
            self.log('EXPERIENCED', prefix='!!')

    def summarize(self):
        text = 'PID {:2d}: {} | API: {} | SOR: {:4d} | TOS: {:2d}mo. | QL: {}'.format(self.id,
                                                            'EXP' if self.is_exp else 'INX',
                                                            self.api_category,
                                                            self.f16_sorties,
                                                            self.tos,
                                                            self.quals)
        if self.ug is not None:
            text += ' | UG: {} #{}'.format(self.ug, self.ride_num)
        return text

class SquadronRoster:
    """A collection of Air Force pilots"""
    
    def __init__(self, sq_name, pilots=[]):
        self.sq_name = sq_name
        self.pilots = pilots
        self.pid = 0

    def add_INX_pilot(self, f16_sorties=0, tos=0, quals=[], ug=None):
        pilot = Pilot(self.pid, f16_sorties, tos, 1, quals, ug)
        self.pilots.append(pilot)
        self.pid += 1
        return pilot

    def add_EXP_pilot(self, f16_sorties=250, tos=20, api_category=1, quals=ug_quals[:-1], ug=None):
        pilot = Pilot(self.pid, f16_sorties, tos, api_category, quals, ug)
        self.pilots.append(pilot)
        self.pid += 1
        return pilot

    def populate(self, size=10, prop_INX=0.55, prop_IP=0.3, num_API6=2):
        num_INX = int(size*prop_INX)
        num_EXP = size - num_INX
        ip_billets_remaining = int(num_EXP*prop_IP)
        num_API1 = size - num_API6
        # Parameters
        max_TOS_INX = 24
        min_TOS_EXP = 24
        max_TOS_EXP = 32

        min_f16_sorties_EXP = 250
        max_f16_sorties_EXP = 500

        max_f16_sorties_INX = 250

        staff_billets_remaining = num_API6
        # Add EXP pilots
        for _ in range(num_EXP):
            tos = rng.integers(min_TOS_EXP, max_TOS_EXP + 1)
            # Award sorties proportional to TOS
            sorties = int((tos - min_TOS_EXP)/(max_TOS_EXP - min_TOS_EXP) *
                      (max_f16_sorties_EXP - min_f16_sorties_EXP)) + min_f16_sorties_EXP

            quals = ug_quals[:-1]

            if staff_billets_remaining > 0:
                api = 6
                staff_billets_remaining -= 1
            else:
                api = 1

            if ip_billets_remaining > 0:
                quals.append(ug_quals[-1])
                ip_billets_remaining -= 1
              
            self.add_EXP_pilot(sorties, tos, api, quals)

        # Add INX pilots
        for _ in range(num_INX):
            tos = rng.integers(max_TOS_INX + 1)
            sorties = int(tos/max_TOS_INX * max_f16_sorties_INX)

            pilot = self.add_INX_pilot(sorties, tos, ug=ug_names[0], quals=[])
            mqt_prog = rng.integers(ug_rides[0]) # Intentionally capping starting INX at 1 MQT ride from end
            for s in range(mqt_prog):
                pilot.fly_ug_sortie()

    def print_sq(self):
        print(f'{self.sq_name} SUMMMARY:')
        for p in self.pilots:
          print(p.summarize())

class F16():
  """A single F-16 jet"""
  hours_between_depot = 500
  depot_days = 90
  mc_rate = 0.7

  def __init__(self, id, num_seats=1, ttaf=2000,
               hours_since_last_depot=500):
    self.id = id
    self.num_seats = num_seats
    self.mc_random_draw()
    self.ttaf = ttaf
    self.hours_since_last_depot = hours_since_last_depot

  def log(self, msg, prefix='>>'):
    print(f'{prefix} {self.id} {msg}')

  def set_nmc(self):
    self.log('is NMC', prefix='<<')
    self.is_mc = False

  def set_mc(self):
    self.log('is MC')
    self.is_mc = True

  def mc_random_draw(self):
    self.is_mc = rng.random() <= self.mc_rate

  def fly_hours(self, hours):
    assert self.is_mc, "Cannot fly NMC aircraft"
    self.ttaf += hours
    self.hours_since_last_depot += hours
    self.log(f'flew {hours} hours. TTAF: {self.ttaf}. Hrs since depot: {self.hours_since_last_depot}', prefix='++')
    if self.needs_depot():
      self.start_depot()

  def sortie(self, duration):
    if self.is_mc:
      self.fly_hours(duration)
    else:
      self.log('not flown due to NMC.', prefix='<X>')

  def needs_depot(self):
    return self.hours_since_last_depot >= self.hours_between_depot

  def start_depot(self):
    self.log('sent to depot!', prefix='!!')
    self.set_nmc()
    self.days_left_in_depot = self.depot_days

  def sit_in_depot(self):
    assert self.days_left_in_depot > 0, "Aircraft already overdue from depot"
    self.days_left_in_depot -= 1

  def complete_depot(self):
    self.log('depot complete.', prefix='**')
    self.set_mc()
    self.days_left_in_depot = -1
    self.hours_since_last_depot = 0

  def print_ac(self):
    print(f'{self.id} | MC?: {self.is_mc} | TTAF: {self.ttaf} | Hrs. since depot: {self.hours_since_last_depot}')


class DailySchedule():
  """Pairing of crews and aircraft """
  pass






In [30]:
ac = F16('3AF')

ac.print_ac()

3AF | MC?: True | TTAF: 2000 | Hrs. since depot: 500


In [38]:
ac.sortie(10)

++ 3AF flew 10 hours. TTAF: 2013.5. Hrs since depot: 10


In [ ]:
sq.populate()
sq.print_sq()

!! PID 0 EXPERIENCED
!! PID 1 EXPERIENCED
!! PID 2 EXPERIENCED
!! PID 3 EXPERIENCED
!! PID 4 EXPERIENCED
>> PID 5 enrolled in MQT
>> PID 6 enrolled in MQT
>> PID 7 enrolled in MQT
>> PID 8 enrolled in MQT
>> PID 9 enrolled in MQT
55 FS SUMMMARY:
PID  0: EXP | API: 6 | SOR:  358 | TOS: 27mo. | QL: ['WM', 'FL', 'IP']
PID  1: EXP | API: 6 | SOR:  483 | TOS: 31mo. | QL: ['WM', 'FL']
PID  2: EXP | API: 1 | SOR:  452 | TOS: 30mo. | QL: ['WM', 'FL']
PID  3: EXP | API: 1 | SOR:  327 | TOS: 26mo. | QL: ['WM', 'FL']
PID  4: EXP | API: 1 | SOR:  358 | TOS: 27mo. | QL: ['WM', 'FL']
PID  5: INX | API: 1 | SOR:  152 | TOS: 13mo. | QL: [] | UG: MQT #2
PID  6: INX | API: 1 | SOR:   31 | TOS:  1mo. | QL: [] | UG: MQT #6
PID  7: INX | API: 1 | SOR:   18 | TOS:  0mo. | QL: [] | UG: MQT #3
PID  8: INX | API: 1 | SOR:  216 | TOS: 19mo. | QL: [] | UG: MQT #4
PID  9: INX | API: 1 | SOR:  193 | TOS: 17mo. | QL: [] | UG: MQT #1


In [ ]:
sq.pilots[8].fly_sortie()

sq.print_sq()

55 FS SUMMMARY:
PID  0: EXP | API: 6 | SOR:  358 | TOS: 27mo. | QL: ['WM', 'FL', 'IP']
PID  1: EXP | API: 6 | SOR:  483 | TOS: 31mo. | QL: ['WM', 'FL']
PID  2: EXP | API: 1 | SOR:  452 | TOS: 30mo. | QL: ['WM', 'FL']
PID  3: EXP | API: 1 | SOR:  327 | TOS: 26mo. | QL: ['WM', 'FL']
PID  4: EXP | API: 1 | SOR:  358 | TOS: 27mo. | QL: ['WM', 'FL']
PID  5: INX | API: 1 | SOR:  152 | TOS: 13mo. | QL: [] | UG: MQT #2
PID  6: INX | API: 1 | SOR:   31 | TOS:  1mo. | QL: [] | UG: MQT #6
PID  7: INX | API: 1 | SOR:   18 | TOS:  0mo. | QL: [] | UG: MQT #3
PID  8: EXP | API: 1 | SOR:  255 | TOS: 19mo. | QL: ['WM', 'FL']
PID  9: INX | API: 1 | SOR:  193 | TOS: 17mo. | QL: [] | UG: MQT #1
